# JUMP ORF data analysis notebook

## Set up environment:
 
1. Create a new environment for this project: `mamba create --name jumpORF python=3.9`

2. Activate that environment `conda activate jumpORF` (note, I'm not sure why but I can't `mamba activate` even after `mamba init` but this seems to work)

3. Install dependencies: 
* `mamba install -c conda-forge dvc-s3` (more instructions here: https://dvc.org/doc/install/macos, specifically you need dvc with aws s3 since this is where the profiles are stored)

## Get data on your local machine:

1. Download the data repo: `git clone https://github.com/jump-cellpainting/jump-orf-data.git` (I use GitHub Desktop for this!)

2. Download the metadata repo: `git clone https://github.com/jump-cellpainting/datasets.git`

3. Pull the files in dvc down to your local computer. In terminal, in the folder where you've cloned `jump-orf-data`: `dvc pull` _(note this step can take a while)_

6. Select the `jumpORF` environment for the kernel for this notebook (upper right of notebook in VScode) or otherwise ensure the jumpORF environment is activated 
   * _note that if you have the notebook open while you make the environment, you may need to restart VScode to see the updated list of environments_


## to do...

2. Use new metadata here as source for finding plates/batches/etc: https://github.com/jump-cellpainting/datasets/tree/main/metadata 
3. Controls include BFP, HcRed, Luciferase, LacZ (but we have excluded eGFP, though it is still showing as a control in the metadata sheet)


## Read in all JUMP ORF data

* Grab the paths to all the profiles from the different batches. 
* Read them into one dataframe (~13000 genes x ~ 5 replicates and a varying # of features depending on whether feature-selected (~1300) or the full data (~5900) profiles are used)

* What do we want to provide? Just the collapsed data? Or the non-collapsed version as well? 
* Is there enough of a reason that we want people to have access to the precollapsed version? 
* Perhaps do not save out these large csvs! Get through cleaning to collapsed data then save that out. 
* concat, collapse, clean as separate function 

In [1]:
# read data
import os
import pandas as pd


#get paths to files using the most truthful metadata
topfolder = "../profiles"
metadata_table = pd.read_csv("../../jump-datasets/metadata/plate.csv",index_col=False)
metadata_table_ORF = metadata_table[(metadata_table.Metadata_PlateType == "ORF")]

batch_list = metadata_table_ORF[(metadata_table_ORF.Metadata_PlateType == "ORF")].Metadata_Batch.unique()

batch_list_2 = metadata_table_ORF.loc[metadata_table_ORF["Metadata_PlateType"] == "ORF"]["Metadata_Batch"].unique()


filesuffix="_normalized_feature_select_negcon_all.csv.gz"
filepaths = [os.path.join(topfolder, metadata_table_ORF.Metadata_Batch.values[row], metadata_table_ORF.Metadata_Plate.values[row],metadata_table_ORF.Metadata_Plate.values[row]+filesuffix) for row in range(len(metadata_table_ORF))]

#only look at files that exist
filepaths = [f for f in filepaths if os.path.exists(f)]

#read in
df = pd.concat(map(lambda file: pd.read_csv(file, index_col=False,), filepaths))

In [2]:
# Get target 2 plates (normalized) and filter the features down to those that are in the df we already have

metadata_table_target2 = metadata_table.loc[(metadata_table["Metadata_Batch"].isin(batch_list)) & (metadata_table["Metadata_PlateType"]=="TARGET2")]

filesuffix="_normalized_negcon.csv" # can also do .csv.gz files
filepaths = [os.path.join(topfolder, metadata_table_target2.Metadata_Batch.values[row], metadata_table_target2.Metadata_Plate.values[row],metadata_table_target2.Metadata_Plate.values[row]+filesuffix) for row in range(len(metadata_table_target2))]

#only look at files that exist
filepaths = [f for f in filepaths if os.path.exists(f)]

#read in
df_t2 = pd.concat(map(lambda file: pd.read_csv(file, index_col=False,), filepaths))

In [6]:
# Filter the target2 df to only columns that exist in the df

t2_metadata_col = [x for x in df_t2.columns if "Metadata" in x]
df_col = list(df.columns)


cols2Keep = list(set(df_col+t2_metadata_col))
cols2Keep = [x for x in cols2Keep if x in list(df_t2.columns)]

df_t2 = df_t2[cols2Keep]

## Filter out ORFs that don't replicate
From Alex: 

Calculate mAP for replicability for each perturbation and filter out those below the random baseline.
The long answer involves the fact that we’ve recently changed what “below random baseline” means. Before, we suggested to subtract mAP of randomly ranked profiles (as suggested in “3.4.2 Computation of the exact random AP value” of my draft on mAP). But recently, we decided that we will consider not mean of random baseline APs, but 95th percentile, such that we can construct a significance test and report p-value instead. Let me know if you want to know more about this, I will also talk about it in special topics on Thursday!

In [6]:
df.head()

,Metadata_plate_map_name,Metadata_broad_sample,Metadata_Plate,Metadata_Well,Metadata_Site_Count,Metadata_Count_Cells,Metadata_Count_CellsIncludingEdges,Metadata_Count_Cytoplasm,Metadata_Count_Nuclei,Metadata_Count_NucleiIncludingEdges,...,Nuclei_Texture_InfoMeas2_Brightfield_3_02_256,Nuclei_Texture_InfoMeas2_Mito_3_02_256,Nuclei_Texture_InverseDifferenceMoment_AGP_3_02_256,Nuclei_Texture_InverseDifferenceMoment_DNA_3_02_256,Nuclei_Texture_InverseDifferenceMoment_Mito_10_03_256,Nuclei_Texture_SumVariance_AGP_10_03_256,Nuclei_Texture_SumVariance_BFHigh_3_03_256,Nuclei_Texture_SumVariance_BFLow_3_00_256,Nuclei_Texture_SumVariance_Brightfield_3_03_256,Nuclei_Texture_SumVariance_ER_10_01_256
0,OAA01.02.03.04.A,ccsbBroad304_05979,BR00117035,A01,9,845,970,845,845,970,...,-0.087364,-0.97526,2.72760,6.7332,3.3709,-1.63900,1.53450,0.690930,1.1301,-2.7376
1,OAA01.02.03.04.A,ccsbBroad304_13129,BR00117035,A02,9,873,988,873,873,988,...,0.198090,-1.85970,1.19280,5.3221,2.9869,-1.13510,6.72200,2.194400,2.5605,-2.2424
2,OAA01.02.03.04.A,ccsbBroad304_00289,BR00117035,A03,9,889,989,889,889,989,...,0.619550,-2.28570,0.54443,3.2157,2.8953,0.21932,4.92190,2.838000,4.3082,-1.5694
3,OAA01.02.03.04.A,ccsbBroad304_99988,BR00117035,A04,9,898,995,898,898,995,...,-0.230090,-0.87500,0.46420,2.5104,2.3601,-0.57261,0.37809,0.672830,1.5367,-1.3152
4,OAA01.02.03.04.A,ccsbBroad304_07679,BR00117035,A05,9,876,982,876,876,982,...,-1.605600,-1.79160,0.64160,2.5965,3.3969,-0.80713,-0.97239,0.032586,1.3970,-2.0045


In [4]:
# feature-select the data

import pycytominer
df_selected = pycytominer.feature_select(df, operation = ['correlation_threshold', 'variance_threshold', 'drop_na_columns', 'blocklist','drop_outliers'], outlier_cutoff = 500)
print('How many columns were dropped?',df.shape[1] - df_selected.shape[1])
df_final = df_selected.loc[:,~df_selected.columns.duplicated()].copy()


How many columns were dropped? 19


In [8]:
df_final.to_parquet(f"JUMP_ORF_all.parquet")

In [9]:
df_parquet = pd.read_parquet(f"JUMP_ORF_all.parquet")

In [8]:
# identify columns with NaN values
[col for col in df_selected.columns if df[col].isnull().values.any()]

['Metadata_broad_sample',
 'Metadata_Name',
 'Metadata_Vector',
 'Metadata_Transcript',
 'Metadata_Symbol',
 'Metadata_NCBI Gene ID',
 'Metadata_Taxon ID',
 'Metadata_Gene Description',
 'Metadata_Annot. Gene Symbol',
 'Metadata_Annot. Gene ID',
 'Metadata_Prot Match %',
 'Metadata_MOI',
 'Metadata_Virus / ml',
 'Metadata_Insert Length',
 'Metadata_pert_type',
 'Metadata_control_type',
 'Cells_AreaShape_FormFactor']

In [6]:
# remove measurement column with NaNs
df_selected.drop(columns='Cells_AreaShape_FormFactor', inplace=True)

In [5]:
# this one does not quite work yet!
import utilitary

# get replicability - setup
replicability_ap_df = pd.DataFrame()
matching_ap_df = pd.DataFrame()

#add metadata_control_type column
all_plates_df = df_selected.copy()
all_plates_df['Metadata_control_type'] = all_plates_df['Metadata_control_type'].fillna('')
# all_plates_df['Metadata_control_type'] = ''
# cmpd = all_plates_df['Metadata_Compound'].values
# ctrl = all_plates_df['Metadata_control_type'].values
# for vals in range(len(cmpd)):
#     if cmpd[vals] == "DMSO":
#         ctrl[vals] = 'negcon'


feature_to_group_by = 'Metadata_Symbol'
# Description
description = f'compound plate'

# Calculate replicability mAP
print(f'Computing {description} replicability...')
precision = utilitary.PrecisionScores(all_plates_df, all_plates_df, feature_to_group_by, "replicability", feature_to_group_by, within=True, against_negcon=True)

replicability_ap_df = precision.ap_group
replicability_map = precision.map

replicability_ap_df.head()
# Construct a random baseline

# Filter the dataframe to only include ORFs that have > 95 percentile of the random baseline (aka, <5% chance of seeing that mAP or something more extreme under the null hypothesis that replicability is random)

Computing compound plate replicability...


ValueError: Input contains NaN.

## Collapse the dataframe within genes

* Median collapse into 1 row per gene (most genes have 5 replicate ORFs) --> data goes down to ~12600 rows
* Metadata_Symbol is the gene name
* Note that the controls include 

In [9]:
# Parameters (to be moved to the top of the notebook)
aggregation_type ="median"


#which control types do you want to include? 
controltypes_orf = ['negcon', 'poscon']
controltypes = ['negcon', 'poscon_cp', 'poscon_orf', 'poscon_diverse']

In [6]:
#filter to gene of interest
df_subset_orf = df.loc[df['Metadata_Symbol'].isin(gene_list)].reset_index(drop=True)

# get controls 
df_subset_orf_con = df.loc[df['Metadata_control_type'].isin(controltypes_orf)].reset_index(drop=True)

#get target 2 data
df_subset_t2 = df_t2.loc[df_t2['Metadata_control_type'].isin(controltypes)].reset_index(drop=True)
df_subset_t2['Metadata_broad_sample'] = df_subset_t2['Metadata_broad_sample'].fillna('empty')

# aggregate
if aggregation_type == "mean":
    df_subset_orf = df_subset_orf.groupby('Metadata_Symbol').mean(numeric_only=True).reset_index(drop=True)
    df_subset_orf_con = df_subset_orf_con.groupby(['Metadata_control_type','Metadata_broad_sample']).mean(numeric_only=True).reset_index(drop=True)
    df_subset_t2 = df_subset_t2.groupby(['Metadata_broad_sample','Metadata_control_type']).mean(numeric_only=True).reset_index(drop=True)

elif aggregation_type == "median":
    df_subset_orf = df_subset_orf.groupby('Metadata_Symbol').median().reset_index(drop=True)
    df_subset_orf_con = df_subset_orf_con.groupby(['Metadata_control_type','Metadata_broad_sample']).median().reset_index(drop=True)
    df_subset_t2 = df_subset_t2.groupby(['Metadata_broad_sample','Metadata_control_type']).median().reset_index(drop=True)

df_subset_orf['Metadata_data_source'] = 'ORF'
df_subset_orf_con['Metadata_data_source'] = 'ORF'
df_subset_t2['Metadata_data_source'] = 'T2'


#merge the separate subsets together
df_subset = pd.concat([df_subset_orf,df_subset_orf_con,df_subset_t2], ignore_index=True)

## for all genes

In [10]:
#fill nas in Metadata_broad_sample column to keep untreated negcons
df_t2['Metadata_broad_sample'] = df_t2['Metadata_broad_sample'].fillna('empty')

# aggregate
if aggregation_type == "mean":
    df_subset_orf = df.groupby('Metadata_Symbol').mean(numeric_only=True).reset_index(drop=True)
    df_subset_orf_con = df.groupby(['Metadata_control_type','Metadata_broad_sample']).mean(numeric_only=True).reset_index(drop=True)
    df_subset_t2 = df_t2.groupby(['Metadata_broad_sample','Metadata_control_type']).mean(numeric_only=True).reset_index(drop=True)

elif aggregation_type == "median":
    df_subset_orf = df.groupby('Metadata_Symbol').median(numeric_only=True).reset_index(drop=True)
    df_subset_orf_con = df.groupby(['Metadata_control_type','Metadata_broad_sample']).median(numeric_only=True).reset_index(drop=True)
    df_subset_t2 = df_t2.groupby(['Metadata_broad_sample','Metadata_control_type']).median(numeric_only=True).reset_index(drop=True)

df_subset_orf['Metadata_data_source'] = 'ORF'
df_subset_orf_con['Metadata_data_source'] = 'ORF'
df_subset_t2['Metadata_data_source'] = 'T2'


#merge the separate subsets together
df_collapsed = pd.concat([df_subset_orf,df_subset_orf_con,df_subset_t2], ignore_index=True)

/var/folders/0y/5d1shp9n6kq856jtm9t10mcw0000gq/T/ipykernel_42906/2054350032.py:11: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_subset_orf = df.groupby('Metadata_Symbol').median().reset_index(drop=True)
/var/folders/0y/5d1shp9n6kq856jtm9t10mcw0000gq/T/ipykernel_42906/2054350032.py:12: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_subset_orf_con = df.groupby(['Metadata_control_type','Metadata_broad_sample']).median().reset_index(drop=True)
/var/folders/0y/5d1shp9n6kq856jtm9t10mcw0000gq/T/ipykernel_42906/2054350032.py:13: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median i

## Put metadata back in the dataframe

In [31]:
import pycytominer

metadata_column_list = ['Metadata_Symbol',
                        'Metadata_control_type', 
                        'Metadata_broad_sample',
                        'Metadata_plate_map_name',
                        'Metadata_Plate', 
                        'Metadata_Name', 
                        'Metadata_Vector',
                        'Metadata_Transcript', 
                        'Metadata_NCBI Gene ID',  
                        'Metadata_Taxon ID',
                        'Metadata_Gene Description',
                        'Metadata_Annot. Gene Symbol',
                        'Metadata_Annot. Gene ID',
                        'Metadata_Prot Match %',
                        'Metadata_MOI',
                        'Metadata_Virus / ml',
                        'Metadata_Insert Length',
                        'Metadata_pert_type',]
#aggregate ORF
df_ORF_aggregated = pycytominer.aggregate(df, 
                        strata=metadata_column_list,
                        features="infer",
                        operation="mean",
                        output_file="none",
                        compute_object_count=False,
                        object_feature="Metadata_ObjectNumber",
                        subset_data_df="none",
                        compression_options=None,
                        float_format=None,)

#aggregate t2 plates from ORF batches


In [28]:
[c for c in df.columns if c not in df_ORF_aggregated.columns]

['Metadata_Well',
 'Metadata_Site_Count',
 'Metadata_Count_Cells',
 'Metadata_Count_CellsIncludingEdges',
 'Metadata_Count_Cytoplasm',
 'Metadata_Count_Nuclei',
 'Metadata_Count_NucleiIncludingEdges',
 'Metadata_Object_Count',
 'Image_Granularity_10_AGP',
 'Image_Granularity_10_BFHigh',
 'Image_Granularity_10_BFLow',
 'Image_Granularity_10_Brightfield',
 'Image_Granularity_10_DNA',
 'Image_Granularity_10_ER',
 'Image_Granularity_10_Mito',
 'Image_Granularity_10_RNA',
 'Image_Granularity_11_AGP',
 'Image_Granularity_11_BFHigh',
 'Image_Granularity_11_BFLow',
 'Image_Granularity_11_Brightfield',
 'Image_Granularity_11_DNA',
 'Image_Granularity_11_ER',
 'Image_Granularity_11_Mito',
 'Image_Granularity_11_RNA',
 'Image_Granularity_12_AGP',
 'Image_Granularity_12_BFHigh',
 'Image_Granularity_12_BFLow',
 'Image_Granularity_12_Brightfield',
 'Image_Granularity_12_ER',
 'Image_Granularity_12_Mito',
 'Image_Granularity_12_RNA',
 'Image_Granularity_13_AGP',
 'Image_Granularity_13_BFHigh',
 'Imag

In [8]:
df_collapsed.to_csv(f"JUMP_ORF_{aggregation_type}_collapsed.csv")